# Bifold_v2
- automodel_nameをGoogle siglip base patchの部分を新しいモデルに変更
- Colab上での表示設定の変更

## GitHubをクローンしてimportできるか

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/syarasyoujyu/bifold.git
%cd bifold

Cloning into 'bifold'...
remote: Enumerating objects: 948, done.
remote: Counting objects: 100% (948/948), done.
remote: Compressing objects: 100% (692/692), done.
remote: Total 948 (delta 227), reused 943 (delta 225), pack-reused 0 (from 0)
Receiving objects: 100% (948/948), 19.42 MiB | 18.35 MiB/s, done.
Resolving deltas: 100% (227/227), done.
/content/bifold


In [ ]:
!pip install .
#!pip install pyflex:こっちのpyfllexとは違うっぽかった

Processing /content/bifold
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 146.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 147.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
import os
os.environ["HF_TOKEN"]=userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"]=userdata.get('WANDB_API_KEY')

In [ ]:
import os
print(f"利用可能なCPUコア数: {os.cpu_count()}")

利用可能なCPUコア数: 12


In [ ]:
%%writefile /content/bifold/bifold/__main__.py

import os
import random
from typing import Dict, Optional, Tuple

import hydra
import matplotlib.pyplot as plt
import numpy as np
import torch
import sys
import wandb
import yaml
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from torchvision.transforms import v2
from tqdm import tqdm      # 表示の変更

from .data import Datasets
from .losses import Losses
from .metrics import Metrics
from .models import Models
from .optim import Optimizers, Schedulers
from .utils.visualization import save_predictions, visualize_action


@hydra.main(version_base=None, config_path="conf", config_name="config")
def main(cfg: DictConfig):
    print(os.getcwd())
    with open("config.yaml", "w") as f:
        OmegaConf.save(cfg, f)

    trainer = Trainer(cfg)

    if not cfg.eval_only:
        trainer.prepare_train()
        trainer.train()
    trainer.eval()


class Trainer:
    def __init__(self, cfg):
        self.device = (
            torch.device("cuda")
            if torch.cuda.is_available() and not cfg.use_cpu
            else torch.device("cpu")
        )

        if cfg.use_wandb and not cfg.eval_only and not cfg.debug:
            self.writer = wandb.init(
                project="WM_Final_v2",
                entity='kazuma_tsuboi',
                group=cfg.train_dataset.name,
                name="+".join(HydraConfig.get().overrides.task),
                resume="allow",
            )
            wandb.config = OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True)
        else:
            self.writer = None

        self.cfg = cfg

        self.seed_randomness()

        self.model = Models.get_by_name(self.cfg.model, device=self.device).to(self.device)

        self.train_dataloader, self.test_dataloader, self.input_processor = (
            Datasets.get_dataloaders(self.cfg)
        )
        self.metrics = Metrics(self.cfg.metrics)

    def train(self):
        if self.start_epoch < self.cfg.epochs:
            for epoch in tqdm(list(range(self.start_epoch, self.cfg.epochs)), desc="Train", leave = False, ncols=75, file=sys.stdout, position=0):   #leave=False
                self.train_epoch(epoch)
                if self.cfg.eval_epochs and (epoch + 1) % self.cfg.eval_epochs == 0:
                    has_improved, _ = self.eval_epoch(epoch)
                    if has_improved:
                        self.save_model(epoch, is_best=True)
                if self.cfg.save_epochs and (epoch + 1) % self.cfg.save_epochs == 0:
                    self.save_model(epoch)

            epoch = self.cfg.epochs - 1
            self.save_model(epoch)

    def eval(self):
        self.load_model(load_best=self.cfg.load_best)
        eval_file = f"eval_{self.cfg.test_dataset.name}.yaml"
        _, metric_dict = self.eval_epoch()
        for k, val in metric_dict.items():
            if isinstance(val, dict):
                for sub_k, sub_val in val.items():
                    print(f"{k} {sub_k}:\t{sub_val:.2f}")
            else:
                print(f"{k}:\t{val:.2f}")

        if os.path.isfile(eval_file):
            print("Found YAML file")
            with open(eval_file, "r") as f:
                old_results = yaml.load(f, Loader=yaml.Loader)
            for k, val in old_results.items():
                if k not in metric_dict:
                    metric_dict[k] = val
                else:
                    if val != metric_dict[k]:
                        print(f"Old value for {k} = {val} ; New value {metric_dict[k]}")
        with open(eval_file, "w") as f:
            yaml.dump(metric_dict, f)

    def seed_randomness(self):
        random.seed(self.cfg.seed)
        np.random.seed(self.cfg.seed)
        torch.manual_seed(self.cfg.seed)
        torch.cuda.manual_seed_all(self.cfg.seed)

    def prepare_train(self):
        params_non_frozen = filter(lambda p: p.requires_grad, self.model.parameters())
        self.loss_fn = Losses.get_by_name(cfg=self.cfg.loss)
        self.optimizer = Optimizers.get_by_name(cfg=self.cfg.optim, params=params_non_frozen)
        assert self.train_dataloader is not None
        self.scheduler = Schedulers.get_by_name(
            cfg=self.cfg.scheduler,
            optimizer=self.optimizer,
            max_iters=len(self.train_dataloader) * self.cfg.epochs,
        )
        self.load_model()

    def train_epoch(self, epoch=None):
        self.model.train()
        total_loss = 0
        pbar = tqdm(self.train_dataloader, desc=f"Train Epoch:{epoch}", leave=False, ncols=75, file=sys.stdout, position=0)     # leave=False
        for i, sample in enumerate(pbar):
            sample = self.move_sample_to_device(sample)

            # Visualize inputs to model
            if self.cfg.debug and self.cfg.visualize_model_inputs:
                self.visualize_model_inputs(sample)

            output = self.model(sample)

            loss, intermediate_losses = self.loss_fn(output, sample)

            self.optimizer.zero_grad()
            assert loss is not None
            loss.backward()

            if self.cfg.debug:
                for name, param in self.model.named_parameters():
                    if param.requires_grad and param.grad is None:
                        raise ValueError(f"Parameter {name} might not have gradient attached!")

            if self.cfg.gradient_clip is not None:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg.gradient_clip)
            self.optimizer.step()
            if self.scheduler is not None:
                self.scheduler.step()

            if self.writer:
                self.writer.log({"loss": loss, "epoch": epoch})
                for k, val in intermediate_losses.items():
                    self.writer.log({k: val})
                for j, param_group in enumerate(self.optimizer.param_groups):
                    self.writer.log({f"lr_{j}": param_group["lr"]})

            assert loss is not None
            total_loss += loss.item()
            # pbar.set_description("Training loss:{}".format(total_loss / (i + 1)))

    def eval_epoch(self, epoch=None) -> Tuple[Optional[bool], Dict[str, torch.Tensor]]:
        self.model.eval()
        has_improved = None
        if epoch is not None or self.cfg.simulator is None:
            # During training or in testing if simulator is not specified
            has_improved, metric_dict = self.eval_epoch_pixel(epoch)
        elif self.cfg.simulator == "softgym":
            if self.cfg.test_dataset.is_bimanual:
                metric_dict = self.eval_epoch_softgym_bimanual(epoch)
            else:
                metric_dict = self.eval_epoch_softgym_single(epoch)
        else:
            raise ValueError(f"Simulator {self.cfg.simulator} not supported")

        if self.writer:
            for k, val in metric_dict.items():
                self.writer.log({k: val})

        if epoch is not None:
            assert (
                has_improved is not None
            ), "has_improved boolean must be specified during training"
        return has_improved, metric_dict

    def eval_epoch_pixel(self, epoch):
        self.metrics.reset()
        num_samples = 0
        for sample in tqdm(self.test_dataloader, desc=f"Evaluate Epoch: {epoch}", leave=False, ncols=75, file=sys.stdout, position=0):     # leave=False
            sample = self.move_sample_to_device(sample)

            # Pick and place prediction
            ret_get_action = self.model.get_action(sample, return_raw_output=True)
            assert isinstance(ret_get_action, Tuple)
            action, raw_output = ret_get_action

            if any("pick" in k for k in sample.keys()):
                # If samples are available
                self.metrics(action=action, sample=sample, raw_output=raw_output)

            if self.cfg.visualize_predictions:
                out_folder = (
                    os.path.join(
                        "eval_background",
                        self.cfg.test_dataset.name,
                        "pixel_metrics",
                        "best" if self.cfg.load_best else "last",
                    )
                    if epoch is None
                    else os.path.join("eval", "pixel_metrics", f"epoch_{epoch}")
                )
                visualizations = visualize_action(sample, action)
                for j in range(len(visualizations)):
                    kwargs = {
                        "rgb": sample["raw_rgb"][j].cpu().numpy(),
                        "depth": sample["depth"][j].squeeze().cpu().numpy(),
                        "viz": visualizations[j],
                    }
                    if raw_output is not None:
                        for k, val in raw_output.items():
                            if "heatmap" in k:
                                kwargs[k] = val[j]
                    save_predictions(
                        out_folder=out_folder,
                        out_file_name=f"{num_samples}_{sample['raw_instruction'][j]}.png",
                        **kwargs,
                    )
                    num_samples += 1

        has_improved, metric_dict = self.metrics.summary()
        if self.writer:
            for k, val in metric_dict.items():
                self.writer.log({k: val})

        return has_improved, metric_dict

    def eval_epoch_softgym_single(self, epoch):
        from .env.softgym_evaluator import SoftgymSingleEvaluator

        evaluator = SoftgymSingleEvaluator(
            cfg=self.cfg,
            model=self.model,
            processor=self.input_processor,
        )
        for task in [
            "CornerFold",
            "TriangleFold",
            "StraightFold",
            "TshirtFold",
            "TrousersFold",
        ]:
            evaluator.evaluate(num_evals=self.cfg.num_evals, task=task)
        evaluator.close()

        return evaluator.summary()

    def eval_epoch_softgym_bimanual(self, epoch):
        from .env.softgym_evaluator import SoftgymBimanualEvaluator

        evaluator = SoftgymBimanualEvaluator(
            self.cfg, model=self.model, processor=self.input_processor
        )
        for i, sample in enumerate(tqdm(self.test_dataloader, desc="Evaluating with SoftGym", leave = False, ncols=75, file=sys.stdout, position=0)):
            evaluator.evaluate(sample)
        evaluator.close()

        return evaluator.summary()

    def load_model(self, load_best=False):
        if load_best and os.path.isfile(os.path.join("checkpoints", "best.pth")):
            checkpoint_file = os.path.join("checkpoints", "best.pth")
        elif os.path.isfile(os.path.join("checkpoints", "last.pth")):
            checkpoint_file = os.path.join("checkpoints", "last.pth")
        else:
            if self.cfg.eval_only and not self.cfg.debug:
                raise FileNotFoundError("Cannot evaluate with untrained model")
            self.start_epoch = 0
            return

        checkpoint = torch.load(checkpoint_file, map_location=self.device, weights_only=False)
        self.start_epoch = checkpoint["epoch"]
        random.setstate(checkpoint["random_states"][0])
        np.random.set_state(checkpoint["random_states"][1])
        torch.set_rng_state(checkpoint["random_states"][2].cpu())
        if torch.cuda.is_available():
            torch.cuda.set_rng_state(checkpoint["random_states"][3].cpu())

        self.model.load_state_dict(checkpoint["model"])

        if not self.cfg.eval_only:
            self.optimizer.load_state_dict(checkpoint["optimizer"])

            if self.scheduler is not None and "scheduler" in checkpoint:
                self.scheduler.load_state_dict(checkpoint["scheduler"])

            if "best_eval" in checkpoint:
                self.metrics.best_eval = checkpoint["best_eval"]

        print(f"Loaded model from checkpoint {checkpoint_file}")

    def save_model(self, epoch, is_best=False):
        os.makedirs("checkpoints", exist_ok=True)
        if is_best:
            model_path = os.path.join("checkpoints", "best.pth")
        else:
            model_path = os.path.join("checkpoints", "last.pth")
        state_dict = {
            "epoch": epoch + 1,
            "random_states": (
                random.getstate(),
                np.random.get_state(),
                torch.get_rng_state(),
                (torch.cuda.get_rng_state() if torch.cuda.is_available() else None),
            ),
            "model": self.model.state_dict(),
            "optimizer": self.optimizer.state_dict(),
        }
        if self.scheduler is not None:
            state_dict["scheduler"] = self.scheduler.state_dict()
        if self.metrics.best_eval is not None:
            state_dict["best_eval"] = self.metrics.best_eval
        torch.save(state_dict, model_path)

    def move_sample_to_device(self, sample):
        # Move all inputs to device
        for k, val in sample.items():
            if isinstance(val, torch.Tensor) or k == "graph":
                sample[k] = val.to(self.device)
        return sample

    def visualize_model_inputs(self, sample):
        input_visualizations = {}
        if self.writer is None:
            fig, axes = plt.subplots(ncols=4, nrows=3)
            flat_axes = axes.flat
            i = 0
        else:
            fig, flat_axes, i = None, None, None

        for k, val in sample.items():
            if any(subk in k for subk in ["rgb", "depth", "heatmap"]):
                if "rgb" in k:
                    transform = v2.Compose([
                        v2.Normalize(
                            (0.0, 0.0, 0.0),
                            (
                                1 / 0.26862954,
                                1 / 0.26130258,
                                1 / 0.27577711,
                            ),
                        ),
                        v2.Normalize(
                            (-0.48145466, -0.4578275, -0.40821073),
                            (1.0, 1.0, 1.0),
                        ),
                        v2.ToPILImage(),
                    ])
                else:
                    transform = v2.ToPILImage()

                if len(val.shape) > 2:
                    if len(val.shape) < 5:
                        imgs = [transform(val[0])]
                        names = [k]
                    else:
                        imgs = [transform(v) for v in val[0]]
                        names = [f"{k}_{i}" for i in range(len(imgs))]

                    for img, name in zip(imgs, names):
                        if self.writer:
                            input_visualizations[name] = wandb.Image(
                                img,
                                caption=sample["raw_instruction"][0],
                            )
                        else:
                            assert flat_axes is not None
                            assert i is not None
                            flat_axes[i].imshow(img)
                            if k == "depth":
                                for k_in, val_in in sample.items():
                                    if "heatmap" in k_in and len(val_in[0].shape) > 1:
                                        flat_axes[i].imshow(transform(val_in[0]), alpha=0.5)
                            flat_axes[i].set_title(name)
                            i += 1

        if self.writer:
            self.writer.log(input_visualizations)
        else:
            assert fig is not None
            fig.suptitle(sample["raw_instruction"][0])
            plt.show()


if __name__ == "__main__":
    main()

Overwriting /content/bifold/bifold/__main__.py


In [ ]:
%%writefile /content/bifold/bifold/conf/config.yaml
---
defaults:
  - _self_
  - model: siglip_sequential
  - dataset@train_dataset: single_sequential
  - dataset@test_dataset: single_sequential   # 変更
  - processor: default
  - loss: bce_gaussmap
  - optim: adam
  - scheduler: linear_warmup
  - override hydra/hydra_logging: disabled
seed: 3407
eval_only: false
eval_epochs: 1  # 毎エポック評価
save_epochs: 1
load_best: true
use_wandb: true    # オン
num_workers: 8
batch_size: 8
gradient_clip: null
test_batch_size: 8
epochs: 30 #num of epoch to train, 30でも十分小さくなっている
debug: false
num_evals: 5 # only used in simulator-only evaluation
metrics:
  computed_metrics: [kp_mse, ap_5, ap_10, ap_20, ap_50, iou, quantile_prob]
  tracked_metric: kp_mse
use_cpu: false
simulator: softgym
dataset_root: /content/drive/MyDrive/東大松尾研究室/7_Thursday_WM_世界モデル/最終課題研究/bifold_datasets/datasets   # ショートカットにする
softgym_cache: ${dataset_root}/tsuboi/softgym_cache
visualize_model_inputs: false
visualize_predictions: false
consecutive: false
hydra:
  output_subdir:
  run:
    dir: /content/drive/MyDrive/東大松尾研究室/7_Thursday_WM_世界モデル/最終課題研究/tsuboi/results_v2
  job:
    chdir: true
    config:
      override_dirname:
        exclude_keys:
          - seed
          - simulator
          - use_wandb
          - eval_only
          - eval_epochs
          - save_epochs
          - dataset
          - dataset_root
          - num_evals
          - softgym_task
          - softgym_cache
          - visualize_attention
          - visualize_actions
          - num_workers
          - load_best
          - test_batch_size
          - visualize_predictions
          - use_cpu
          - debug

Overwriting /content/bifold/bifold/conf/config.yaml


In [ ]:
%%writefile /content/bifold/bifold/conf/model/siglip_sequential.yaml
---
name: siglip_sequential
image_size: ${train_dataset.image_size}
is_bimanual: ${train_dataset.is_bimanual}
patch_size: 16
automodel_name: google/siglip2-base-patch${model.patch_size}-${model.image_size}    # 変更点
dim: 768
emb_dropout: 0.0
lora: true
r: 8
lora_alpha: 32
lora_dropout: 0.01
target_modules: [q_proj, v_proj]
threshold: 0.5
text_encoder: null
pick_place_model: pick_place_convdecoder
fusion_model: concat_transformer
depth: 8
heads: 16
mlp_ratio: 4
dropout: 0.0
context_length: ${train_dataset.max_context_length}
use_pos_embed: false
text_max_length: 64

Overwriting /content/bifold/bifold/conf/model/siglip_sequential.yaml


In [ ]:
%cd /content/bifold
!python -m bifold

/content/bifold
/content/drive/MyDrive/東大松尾研究室/7_Thursday_WM_世界モデル/最終課題研究/tsuboi/results_v2
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: chem-tsuboi to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: ⣽ Waiting for wandb.init()...
wandb: ⣾ setting up run o48s6pbb (0.3s)
wandb: ⣷ setting up run o48s6pbb (0.3s)
wandb: ⣯ setting up run o48s6pbb (0.3s)
wandb: Tracking run with wandb version 0.24.2
wandb: Run data is saved locally in /content/drive/MyDrive/東大松尾研究室/7_Thursday_WM_世界モデル/最終課題研究/tsuboi/results_v2/wandb/run-20260214_113514-o48s6pbb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run smoldering-infatuation-9
wandb: ⭐️ View project at https://wandb.ai/kazuma_tsuboi/WM_Final_v2
wandb: 🚀 View run at https://wandb.ai/kazuma_tsuboi/WM_Final_v2/runs/o48s6pbb
[2026-02-14 11:35:17,927][httpx][INFO] - HT

In [ ]:
%cd /content/bifold
!python -m bifold eval_only=true simulator=null #lossだけ確認

/content/bifold
/content/drive/MyDrive/東大松尾研究室/7_Thursday_WM_世界モデル/最終課題研究/tsuboi/results_v2
[2026-02-14 12:45:29,113][httpx][INFO] - HTTP Request: HEAD https://huggingface.co/google/siglip2-base-patch16-224/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
[2026-02-14 12:45:29,154][httpx][INFO] - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/siglip2-base-patch16-224/75de2d55ec2d0b4efc50b3e9ad70dba96a7b2fa2/config.json "HTTP/1.1 200 OK"
[2026-02-14 12:45:29,238][httpx][INFO] - HTTP Request: HEAD https://huggingface.co/google/siglip2-base-patch16-224/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
[2026-02-14 12:45:29,278][httpx][INFO] - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/siglip2-base-patch16-224/75de2d55ec2d0b4efc50b3e9ad70dba96a7b2fa2/config.json "HTTP/1.1 200 OK"
Loading weights: 100% 408/408 [00:00<00:00, 479.24it/s, Materializing param=vision_model.post_layernorm.weight]
trainable params: 589824 